In [1]:
import pandas as pd
import plotly.graph_objects as go
from scipy.optimize import differential_evolution, NonlinearConstraint

from utils import gen_data
from optimum import objective

import numpy as np

from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args


In [2]:
target_dat = gen_data("C:\\Users\\miche\\Downloads\\trade-log-1030-4.csv.csv")

c:\Users\miche\OptionsAnalysis\utils.py:61: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  target_dat = pd.read_csv(
c:\Users\miche\OptionsAnalysis\utils.py:61: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  target_dat = pd.read_csv(


In [4]:
bounds = [
    (target_dat.VIX_ON_CHANGE.min(), target_dat.VIX_ON_CHANGE.max() - 0.01 ),  # VIX_overnight_min
    (target_dat.VIX_ON_CHANGE.min() + 0.01, target_dat.VIX_ON_CHANGE.max()),  # VIX_overnight_max
    (target_dat.SPX_ON_CHANGE.min(), target_dat.SPX_ON_CHANGE.max() -0.01),  # SPX_overnight_min
    (target_dat.SPX_ON_CHANGE.min() + 0.01 , target_dat.SPX_ON_CHANGE.max()),  # SPX_overnight_max
    (target_dat.VIX_CURRENT_CLOSE.min(), target_dat.VIX_CURRENT_CLOSE.max()-1),  # VIX_prior_min
    (target_dat.VIX_CURRENT_CLOSE.min()+1, target_dat.VIX_CURRENT_CLOSE.max())  # VIX_prior_max
]

# Define constraints to ensure min is less than max for each range using NonlinearConstraint
constraints = [
    NonlinearConstraint(lambda x: x[1] - x[0], 0, np.inf),  # VIX_overnight_max > VIX_overnight_min
    NonlinearConstraint(lambda x: x[3] - x[2], 0, np.inf),  # SPX_overnight_max > SPX_overnight_min
    NonlinearConstraint(lambda x: x[5] - x[4], 0, np.inf)   # VIX_prior_max > VIX_prior_min
]


# Define initial guess (midpoints of bounds)
x0 = [-0.05, 0.05, -0.01, 0.01, 15, 20]


# Define the parameter space for Bayesian Optimization
space  = [
    Categorical(['best1bin', 'rand1bin', 'best2bin', 'rand2bin'], name='strategy'),
    Integer(10, 20, name='popsize'),  # Population size multiplier
    Real(0.5, 1.5, name='mutation_low'),  # Lower bound of mutation factor
    Real(0.5, 1.5, name='mutation_high'),  # Upper bound of mutation factor
    Real(0.5, 0.9, name='recombination')  # Recombination constant
]



# Decorate the objective function to use named arguments for the hyperparameters
@use_named_args(space)
def skopt_objective(**params):
    print(f"Testing parameters: {params}")
    
    # Perform optimization with the current set of hyperparameters
    result = differential_evolution(
        objective,
        bounds = bounds,
        args=(target_dat,50),
        strategy=params['strategy'],
        maxiter=1000,
        popsize=params['popsize'],
        tol=1e-6,
        mutation=(params['mutation_low'], params['mutation_high']),
        recombination=params['recombination'],
        seed=42,
        disp=False,
        constraints=constraints,
        polish=True  # Use local search for polishing
    )
    
    # Return negative WIN_RATE as objective function value (because skopt minimizes)
    return result.fun

# Run Bayesian Optimization
res_gp = gp_minimize(
    skopt_objective,  # the function to minimize
    space,  # the parameter space
    n_calls=20,  # the number of evaluations of the objective function
    random_state=42,  # seed for reproducibility
    acq_func="EI",  # acquisition function (Expected Improvement)
    n_initial_points=5  # number of random points to start with
)

# Print the best hyperparameters
print("\nBest Hyperparameters Found:")
print(dict(zip([p.name for p in space], res_gp.x)))
print("Best Objective Function Value (WIN_RATE):", -res_gp.fun)  # Negative because we minimize




Testing parameters: {'strategy': 'rand2bin', 'popsize': np.int64(12), 'mutation_low': 1.2796910002727695, 'mutation_high': 1.096850157946487, 'recombination': 0.6783331011414365}


C:\Users\miche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\optimize\_differentiable_functions.py:316: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
C:\Users\miche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\optimize\_differentiable_functions.py:551: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


Testing parameters: {'strategy': 'best1bin', 'popsize': np.int64(15), 'mutation_low': 0.833708611139022, 'mutation_high': 0.6428668179219408, 'recombination': 0.7603553891795412}
Testing parameters: {'strategy': 'best1bin', 'popsize': np.int64(17), 'mutation_low': 1.4385527090157504, 'mutation_high': 0.5007787658410143, 'recombination': 0.8968846237164871}
Testing parameters: {'strategy': 'best2bin', 'popsize': np.int64(16), 'mutation_low': 0.5070663052197174, 'mutation_high': 0.5230624250414158, 'recombination': 0.7099098641033557}
Testing parameters: {'strategy': 'rand1bin', 'popsize': np.int64(10), 'mutation_low': 1.4737555188414593, 'mutation_high': 0.7327713404303042, 'recombination': 0.5362425738131283}
Testing parameters: {'strategy': np.str_('best1bin'), 'popsize': np.int64(20), 'mutation_low': 1.5, 'mutation_high': 1.5, 'recombination': 0.9}


C:\Users\miche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\optimize\_differentiable_functions.py:316: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
C:\Users\miche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\optimize\_differentiable_functions.py:551: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


Testing parameters: {'strategy': np.str_('rand1bin'), 'popsize': np.int64(10), 'mutation_low': 1.5, 'mutation_high': 0.5, 'recombination': 0.5}


...

Testing parameters: {'strategy': np.str_('rand1bin'), 'popsize': np.int64(17), 'mutation_low': 1.4189262037758188, 'mutation_high': 0.6332901711836948, 'recombination': 0.6778865466993305}


C:\Users\miche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\optimize\_differentiable_functions.py:316: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
C:\Users\miche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\optimize\_differentiable_functions.py:551: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


Testing parameters: {'strategy': np.str_('best2bin'), 'popsize': np.int64(20), 'mutation_low': 0.5484116580275505, 'mutation_high': 1.4703110373197041, 'recombination': 0.895396826137137}


C:\Users\miche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\optimize\_differentiable_functions.py:316: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
C:\Users\miche\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\scipy\optimize\_differentiable_functions.py:551: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)



Best Hyperparameters Found:
{'strategy': 'best1bin', 'popsize': np.int64(17), 'mutation_low': 1.4385527090157504, 'mutation_high': 0.5007787658410143, 'recombination': 0.8968846237164871}
Best Objective Function Value (WIN_RATE): 594.5645
